In [129]:
import pandas as pd

In [130]:
import numpy as np

## 1. Data Loadind

#### Raw data parsed from https://www.resultados.com/futebol/brasil/serie-a/, transformed into a spreadsheet and expoted to a CSV file.

In [131]:
jogos = pd.read_csv("data/brasileiro-a.9.csv")

In [132]:
jogos.head()

,campeonato,ano,rodada,data,mandante,visitante,placar
0,brasileiro-a,2018,9,04.06. 20:00,Paraná,Fluminense,2 : 1
1,brasileiro-a,2018,9,03.06. 19:00,Ceará,Cruzeiro,0 : 1
2,brasileiro-a,2018,9,03.06. 16:00,América-MG,Atlético-PR,3 : 1
3,brasileiro-a,2018,9,03.06. 16:00,Bahia,Grêmio,0 : 2
4,brasileiro-a,2018,9,03.06. 16:00,Flamengo,Corinthians,1 : 0


In [133]:
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6214 entries, 0 to 6213
Data columns (total 7 columns):
campeonato    6214 non-null object
ano           6214 non-null int64
rodada        6214 non-null int64
data          6214 non-null object
mandante      6214 non-null object
visitante     6214 non-null object
placar        6214 non-null object
dtypes: int64(2), object(5)
memory usage: 339.9+ KB


## 2. Data Parsing 

### 2.1 Remove white spaces

In [134]:
# select columns of type object
jogos_obj = jogos.select_dtypes(['object'])

In [135]:
jogos[jogos_obj.columns] = jogos_obj.apply(lambda x: x.str.strip())

### 2.2 List soccer teams from all seasons

#### 2.2.1 Get unique mandatory teams 

In [136]:
times = list(jogos["mandante"].unique())

#### 2.2.2 Sort teams by name

In [137]:
times.sort()

#### 2.2.3 Check the teams

In [138]:
times

['América-MG',
 'América-RN',
 'Atlético-GO',
 'Atlético-MG',
 'Atlético-PR',
 'Avaí',
 'Bahia',
 'Barueri',
 'Botafogo',
 'Brasiliense',
 'Ceará',
 'Chapecoense',
 'Corinthians',
 'Coritiba',
 'Criciúma',
 'Cruzeiro',
 'Figueirense',
 'Flamengo',
 'Fluminense',
 'Fortaleza',
 'Goias',
 'Grêmio',
 'Guarani',
 'Internacional',
 'Ipatinga',
 'Joinville',
 'Juventude',
 'Náutico',
 'Palmeiras',
 'Paraná',
 'Paysandu',
 'Ponte Preta',
 'Portuguesa',
 'Santa Cruz',
 'Santo Andre',
 'Santos',
 'Sao Caetano',
 'Sport Recife',
 'São Paulo',
 'Vasco',
 'Vitória']

#### 2.2.4 Check the number of times

In [139]:
len(times)

41

### 2.3 Transform score in two columns

#### 2.3.1 Defines a function to parse the score

In [140]:
# Function returns first part of score if is the home team or second part otherwise
def tratarPlacar(row, mandante):
    placar = row["placar"].split(" : ")
    if (mandante):
        return int(placar[0])
    return int(placar[1])

#### 2.3.2 Fills the goals of the home team in to a column

In [141]:
jogos['gols_mandante'] = jogos.apply (lambda row: tratarPlacar(row, True),axis=1)

#### 2.3.2 Fills the goals of the visitor team in to a column

In [142]:
jogos['gols_visitante'] = jogos.apply (lambda row: tratarPlacar(row, False),axis=1)

#### 2.3.3 Remove scrore column

In [143]:
del jogos["placar"]

#### 2.3.4 Describe the matches DataFrame

In [144]:
jogos.describe()

,ano,rodada,gols_mandante,gols_visitante
count,6214.000000,6214.000000,6214.000000,6214.000000
mean,2009.687480,20.154168,1.600579,1.048439
std,4.540624,11.673962,1.253268,1.041117
min,2003.000000,1.000000,0.000000,0.000000
25%,2005.000000,10.000000,1.000000,0.000000
50%,2010.000000,20.000000,1.000000,1.000000
75%,2014.000000,30.000000,2.000000,2.000000
max,2018.000000,46.000000,7.000000,7.000000


#### 2.3.5 Check the matches DataFrame

In [145]:
jogos.head(2)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante
0,brasileiro-a,2018,9,04.06. 20:00,Paraná,Fluminense,2,1
1,brasileiro-a,2018,9,03.06. 19:00,Ceará,Cruzeiro,0,1


### 2.4 Parse the date column

#### 2.4.1 Defines a function to parse the date

In [146]:
# Converts the date from dd.mm to dd-mm format
def tratarData(row):
    data = row["data"].split(".")    
    return data[0] + "-" + data[1]

#### 2.4.2 Parse data form "data" column

In [147]:
jogos['data'] = jogos.apply (lambda row: tratarData(row),axis=1)

#### 2.4.3 Check the result on  "data"column 

In [148]:
jogos.head(2)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante
0,brasileiro-a,2018,9,04-06,Paraná,Fluminense,2,1
1,brasileiro-a,2018,9,03-06,Ceará,Cruzeiro,0,1


### 2.5 Create a result column 

#### 2.5.1 Defines a function to parse score

In [149]:
# Retuns "MANDANTE" if the home team wins the macth,
# "VISITANTE" if visitor wins or "EMPATE" in case of tied match
def resultado(row):
    if (row["gols_mandante"] > row["gols_visitante"]):
        return "MANDANTE"
    elif (row["gols_mandante"] < row["gols_visitante"]):
        return "VISITANTE"
    else:
        return "EMPATE"

#### 2.5.2 Apply the function to "resultado" column

In [150]:
jogos['resultado'] = jogos.apply (lambda row: resultado(row),axis=1)

#### 2.5.3 Check the matches DataFrame

In [151]:
jogos.head(2)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado
0,brasileiro-a,2018,9,04-06,Paraná,Fluminense,2,1,MANDANTE
1,brasileiro-a,2018,9,03-06,Ceará,Cruzeiro,0,1,VISITANTE


### 2.6 Export matches DataFrame to file

In [152]:
jogos.to_csv("data/match-results.9.csv", sep=",")